In [1]:
import torch
import torch_geometric
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/home/asphyxia/anaconda3/envs/tensorflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("required_format.csv")
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,primary_channel,min_channel_allowed,max_channel_allowed,rssi,node_type,sinr,air_time_mean,deployment,channel_bonding_model,throughput
0,0,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,...,0,0,3,-55.42,0,35.68,25.15,0.0,4,104.96
1,1,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,...,0,0,3,-62.31,1,26.99,25.15,0.0,4,7.68
2,2,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,...,0,0,3,-55.42,1,35.68,25.15,0.0,4,11.09
3,3,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,...,0,0,3,-58.23,1,33.42,25.15,0.0,4,14.51
4,4,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,...,0,0,3,-66.64,1,26.83,25.15,0.0,4,11.95


In [3]:
def get_int_map(dep):
    dep = df.loc[df["deployment"]==dep]
    dep = dep.reset_index(drop=True)
    return dep

In [4]:
df.columns

Index(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', 'wlan_code_index', 'x(m)', 'y(m)', 'z(m)', 'primary_channel',
       'min_channel_allowed', 'max_channel_allowed', 'rssi', 'node_type',
       'sinr', 'air_time_mean', 'deployment', 'channel_bonding_model',
       'throughput'],
      dtype='object')

## VERSION 1

In [79]:
distance = pd.read_csv("GNN_data.csv")

df["distance"] = distance["distance"]

In [80]:
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,min_channel_allowed,max_channel_allowed,rssi,node_type,sinr,air_time_mean,deployment,channel_bonding_model,throughput,distance
0,0,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,...,0,3,-55.42,0,35.68,25.15,0.0,4,104.96,NaN
1,1,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,...,0,3,-62.31,1,26.99,25.15,0.0,4,7.68,6.693461
2,2,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,...,0,3,-55.42,1,35.68,25.15,0.0,4,11.09,3.931072
3,3,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,...,0,3,-58.23,1,33.42,25.15,0.0,4,14.51,4.937439
4,4,0.0,-79.65,-93.86,-109.01,-78.68,-85.3,-98.24,-108.79,-97.21,...,0,3,-66.64,1,26.83,25.15,0.0,4,11.95,8.944840


In [108]:
def create_graph(split,split_y,deployment):
    dep = get_int_map(deployment)
    dep_y = dep["throughput"]
    dep_x = dep[['0', '1', '2', '3', '4', '5', '6', '7','8', '9', '10', '11', 'wlan_code_index', 'x(m)', 'y(m)','z(m)',
            'primary_channel', 'min_channel_allowed', 'max_channel_allowed', 'rssi', 'node_type',
            'sinr', 'air_time_mean', 'deployment',"channel_bonding_model"]]
    dep_ap = dep[dep["node_type"]==0]
    dep_ap.reset_index(inplace=True)
    dep_ap.drop(["Unnamed: 0"],axis=1,inplace=True)
    dep_reset = dep.reset_index(drop=True)
    ap_index = {}
    out = dep_reset[dep_reset["node_type"] == 0]
    for i in range(len(out)):
        ap_index[out.index[i]] = i
    #append values of dep_x row from the 12th column to the end only when node_type is 0
    ap_node_features = dep_x[dep_x["node_type"]==0].iloc[:,12:].values
    #convert sinr,rssi in ap_node_features to 0
    ap_node_features[:,7] = 0
    ap_node_features[:,9] = 0
    sta_node_features = dep_x.iloc[:,12:].values
    #drop the second last column from sta_node_features and ap_node_features
    sta_node_features = np.delete(sta_node_features,11,1)
    sta_node_features = np.delete(sta_node_features,0,1)
    ap_node_features = np.delete(ap_node_features,11,1)
    ap_node_features = np.delete(ap_node_features,0,1)
    
    ap_node_features = torch.tensor(ap_node_features,dtype=torch.float)
    sta_node_features = torch.tensor(sta_node_features,dtype=torch.float)
   
    #0 for STA-AP, 1 for AP-AP
    #edge features = [edge_type,distance,rssi,interference]
    
    edges = []
    edge_features = []
    edge_index = []
    for idx,val in dep_ap.iterrows():
        for j,valj in dep_ap.iterrows():
            i_pos = np.asarray([dep_ap.at[idx,"x(m)"],dep_ap.at[idx,"y(m)"],dep_ap.at[idx,"z(m)"]])
            j_pos = np.asarray([dep_ap.at[j,"x(m)"],dep_ap.at[j,"y(m)"],dep_ap.at[j,"z(m)"]])
            # dep.at[val["index"],f"AP_D{j}"] = np.linalg.norm(i_pos - j_pos)
            distance = np.linalg.norm(i_pos - j_pos)
            if idx != j:
                edges.append([dep_ap.at[idx,"index"],dep_ap.at[j,"index"]])
                edge_features.append([1,distance,0,dep_ap.at[idx,str(j)]])
                # edge_index.append([ap_index[idx],ap_index[j]])

    dep_sta = dep[dep["node_type"]==1]
    for idx,val in dep_sta.iterrows():
        edges.append([idx,dep_ap.at[dep_sta.at[idx,"wlan_code_index"],"index"]])
        edge_features.append([0,dep_sta.at[idx,"distance"] ,dep_sta.at[idx,"rssi"],0])


    # for i in range(len(dep)):
    #     for j in range(len(dep)):
    #         if (i != j and (dep["node_type"].iloc[i] == 0 and dep["node_type"].iloc[j] == 0)) or (i !=j and (dep["node_type"].iloc[i] == 1 and dep["node_type"].iloc[j] == 0)):
    #             edges.append([i,j])
    print(edges)
    print(edge_features)



In [109]:
create_graph(0,0,0)

[[0, 11], [0, 27], [0, 44], [0, 57], [0, 70], [0, 82], [0, 97], [0, 113], [0, 128], [0, 139], [0, 156], [11, 0], [11, 27], [11, 44], [11, 57], [11, 70], [11, 82], [11, 97], [11, 113], [11, 128], [11, 139], [11, 156], [27, 0], [27, 11], [27, 44], [27, 57], [27, 70], [27, 82], [27, 97], [27, 113], [27, 128], [27, 139], [27, 156], [44, 0], [44, 11], [44, 27], [44, 57], [44, 70], [44, 82], [44, 97], [44, 113], [44, 128], [44, 139], [44, 156], [57, 0], [57, 11], [57, 27], [57, 44], [57, 70], [57, 82], [57, 97], [57, 113], [57, 128], [57, 139], [57, 156], [70, 0], [70, 11], [70, 27], [70, 44], [70, 57], [70, 82], [70, 97], [70, 113], [70, 128], [70, 139], [70, 156], [82, 0], [82, 11], [82, 27], [82, 44], [82, 57], [82, 70], [82, 97], [82, 113], [82, 128], [82, 139], [82, 156], [97, 0], [97, 11], [97, 27], [97, 44], [97, 57], [97, 70], [97, 82], [97, 113], [97, 128], [97, 139], [97, 156], [113, 0], [113, 11], [113, 27], [113, 44], [113, 57], [113, 70], [113, 82], [113, 97], [113, 128], [113, 

/tmp/ipykernel_3954/3637101600.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dep_ap.drop(["Unnamed: 0"],axis=1,inplace=True)
